<div style="position: relative; background: linear-gradient(135deg, #24398A 0%, #1a2a66 100%); border-radius: 20px 20px 0px 0px; padding: 30px;  box-shadow: 0 8px 16px rgba(36, 57, 138, 0.3);">
  
  
  <div style="position: absolute; top: 20px; right: 20px; background: white; padding: 4px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.2);">
    <img src="images/logo_unison.jpg" alt="UNISON" style="height: 160px;">
  </div>
  
  
  <div style="position: absolute; top: 20px; left: 20px; background: white; padding: 4px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.2);">
    <img src="images/genetic_banner.jpg" alt="Curso" style="height: 160px;">
  </div>
   
  
  <div style="color: white; max-width: 60%; margin: 0 auto; text-align: center;">
    <h1 style="color: #EBA93B; margin: 0; font-size: 28px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
      Metaheurísticas para Ciencia de Datos: <br>Teoría y Práctica
    </h1><br>
    <h3 style="margin: 15px 0 5px 0; font-size: 16px; opacity: 0.95;">Maestría en Ciencia de Datos</br></br>
    Ramón Soto C. / ramon.soto@unison.mx</h3>
  </div>
</div>

<div style="background: white; border: 3px solid #EBA93B; border-radius: 0px 0px 20px 20px; padding: 25px;">
  <div style="display: flex; align-items: center; margin-bottom: 15px;">
    <div style="background: #24398A; color: white; font-size: 24px; font-weight: bold; padding: 10px 20px; border-radius: 8px; margin-right: 20px;">02</div>
    <div>
      <h2 style="color: #24398A; margin: 0;">Sistemas Complejos y la Naturaleza Compleja de las Series de Tiempo</h2>
      <p style="color: #666; margin: 5px 0 0 0; font-style: italic;">Una mirada al mundo real, más allá de las funciones lisas y los modelos dóciles</p>
    </div>
  </div>
  
  <div style="background: #f8f9fa; padding: 15px; border-radius: 5px; border-left: 4px solid #24398A;">
    <h4><strong>Presentación general:</strong></h4>
    <p>La ciencia de datos tradicional asume mundos lineales y estacionarios. Pero ¿qué ocurre cuando los datos provienen de sistemas dinámicos no lineales, caóticos o con propiedades emergentes? Esta libreta nos introduce al mundo de los sistemas complejos a través del estudio de series temporales reales. Aprenderemos a reconocer comportamientos caóticos, atractores extraños y dinámicas no lineales que invalidan los supuestos de métodos estadísticos convencionales. Mediante el estudio de sistemas como Lorenz y Mackey-Glass, desarrollaremos una "humildad epistemológica" frente a la complejidad del mundo real, al tiempo que identificaremos estructuras subyacentes que pueden ser capturadas con herramientas apropiadas.
</p>
  </div>
</div>


<div style="padding: 15px 20px; background-color: #f8f9fa; margin: 20px 0;">
    <p style="font-size: 16px; line-height: 1.6; color: #333; margin: 0;">
        En dominios como los mercados financieros o la dinámica atmosférica, no existe una dinámica estable que permita modelar el sistema mediante la reducción a sus partes individuales. Los <b>sistemas complejos</b> se caracterizan por la emergencia de patrones globales a partir de interacciones locales simples. Para el científico de datos, esto implica reconocer que la linealidad es una excepción y que el verdadero reto reside en capturar la auto-organización y la sensibilidad a condiciones iniciales que definen el comportamiento de los datos en entornos reales.
    </p>
</div>

In [ ]:
# CONFIGURACIÓN INICIAL
# =====================

# Instalar librerías no estándar
# nolds: para análisis no lineal (Lyapunov, D2)
# yfinance: para datos financieros reales
!pip install -q nolds yfinance

# Configuración inicial
import numpy as np
from scipy.integrate import odeint
import scipy.stats as stats
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller
import nolds
import yfinance as yf

import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import pandas as pd
from collections import deque

# Configuración estética
np.set_printoptions(precision=4, suppress=True)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12

# Reproducibilidad
np.random.seed(42)

print("Librerías cargadas exitosamente.\n")

# 1. La Complejidad como Condición Natural del Mundo

Cuando hablamos de ciencia de datos, solemos imaginar modelos aplicados a conjuntos ordenados de información, donde los fenómenos siguen patrones discernibles, regularidades estables y estructuras razonablemente predecibles. Pero el mundo real rara vez se comporta así.

Gran parte de los fenómenos que estudiamos en economía, biología, física, energía, clima, neurología y sistemas sociales **no son lineales ni estacionarios**. Son sistemas donde millones de microsucesos interactúan constantemente, dando lugar a comportamientos globales que no pueden deducirse de sus partes.

A esa familia de fenómenos la llamamos **sistemas complejos**.

Y una de sus huellas más frecuentes son las **series de tiempo complejas**.

---

## PARTE 1: FUNDAMENTOS TEÓRICOS

### 2. ¿Qué es un Sistema Complejo?

Un sistema complejo no es simplemente un sistema "complicado".

Es un sistema donde:
* ✅ **Múltiples componentes interactúan localmente** (sin coordinador central)
* ✅ **Pequeñas variaciones $\rightarrow$ grandes efectos** (sensibilidad a condiciones iniciales)
* ✅ **Estructuras globales emergen** sin diseño explícito (**emergencia**)
* ✅ **Comportamiento multimodal** (estable o caótico según contexto)
* ✅ **El ruido puede ser funcional**, no error de medición
* ✅ **Los modelos clásicos fallan** (lineales, gaussianos, estacionarios)

| Aspecto | Sistema Complicado | Sistema Complejo |
| :--- | :--- | :--- |
| Ejemplo | Motor de avión | Ecosistema, cerebro |
| Componentes | Muchos, pero independientes | Pocos, pero altamente acoplados |
| Comportamiento | Predecible dadas las partes | Emergente, no deducible |
| Análisis | Reduccionismo funciona | Reduccionismo falla |
| Causalidad | Lineal, trazable | No lineal, circular |
| Ruido | Error de medición | Parte funcional del sistema |
| Modelado | Ingeniería clásica | Teoría de sistemas dinámicos |

**Ejemplos Paradigmáticos**

* **🌍 Sistemas Físicos:** Clima global, Turbulencia en fluidos, Redes eléctricas.
* **🧠 Sistemas Biológicos:** El cerebro (86 mil millones de neuronas), El corazón (arritmias), Redes de regulación genética.
* **💰 Sistemas Socio-Económicos:** Mercados financieros, Tráfico vehicular urbano, Pandemias (COVID-19).

**Característica común:** Son impredecibles en detalle, pero predecibles en estructura (atractores, patrones estadísticos).

### 2.1 Propiedades Emergentes: El Todo es Más que la Suma

Una propiedad **emergente** es un comportamiento del sistema completo que no existe en ningún componente individual.

**Ejemplos:**

* **Conciencia** emerge de neuronas (ninguna neurona individual es consciente)
* **Tráfico congestionado** emerge de conductores individuales (ninguno lo planeó)
* **Crashes financieros** emergen de traders racionales (nadie los diseña)
* **Patrones de enjambre** emergen de reglas locales simples

**Implicación para ML:** No puedes entender el sistema estudiando variables de forma aislada (correlación univariada). Necesitas capturar interacciones no lineales.

---

In [ ]:
# [CÓDIGO 1: Visualización de emergencia (Simulación de Boids)]

print("### Simulación de Comportamiento Emergente (Boids) ###")
print("Este código simularía la emergencia de un patrón de 'bandada' a partir de 3 reglas locales simples:")
print("1. Separación (evitar vecinos), 2. Alineación (igualar velocidad), 3. Cohesión (moverse hacia el centro de masa).")
print("La estructura global de la bandada emerge sin un líder central.")

# Se requiere una implementación de Boids/Flocking. Por razones de concisión en JSON,
# solo se incluye la estructura del código y un ejemplo simple de visualización de puntos aleatorios para demostrar el espacio.

N_Boids = 50
coords = np.random.rand(N_Boids, 2) * 10

fig, ax = plt.subplots()
scat = ax.scatter(coords[:, 0], coords[:, 1], color='blue', s=10)
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.set_title("Estructura para la Simulación de Boids (Emergencia)")
plt.show()

### 3. La Firma Matemática: Caos, Atractores y Sensibilidad

Los sistemas complejos suelen presentar **caos determinista**.

Esto **NO significa "ruido"**, sino:

Dinámicas gobernadas por ecuaciones exactas que producen comportamientos aparentemente aleatorios.

#### 3.1 El Atractor de Lorenz: El Icono del Caos

Edward Lorenz (1963) descubrió el caos determinista al simplificar ecuaciones de convección atmosférica:

$$
\begin{aligned}
\dot{x} &= \sigma(y - x) \\
\dot{y} &= x(\rho - z) - y \\
\dot{z} &= xy - \beta z
\end{aligned}
$$

Parámetros típicos: $\sigma = 10$, $\rho = 28$, $\beta = 8/3$

**Propiedades:**

* **Sensibilidad extrema:** Dos trayectorias cercanas divergen exponencialmente
* **Atractor extraño:** Forma una mariposa en 3D (fractal, dimensión no entera)
* **No periódico:** Nunca se repite exactamente
* **Determinista:** Mismas condiciones iniciales $\rightarrow$ misma trayectoria

**Frase histórica de Lorenz:**

> "Un aleteo de mariposa en Brasil puede causar un tornado en Texas."

Esto no es poético: es matemáticamente riguroso. Pequeños errores de medición se amplifican exponencialmente.

---

In [ ]:
# [CÓDIGO 2: Sistema de Lorenz]

def lorenz_system(state, t, sigma, rho, beta):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Parámetros clásicos
sigma = 10
rho = 28
beta = 8/3

# Condiciones iniciales
initial_state = [0.0, 1.0, 1.05]
t = np.arange(0, 50, 0.01)

# Resolver EDO
trajectory = odeint(lorenz_system, initial_state, t, args=(sigma, rho, beta))

### 1. Visualización del atractor 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], lw=0.5, color='blue')
ax.set_title("Atractor Extraño de Lorenz (3D)")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.show()

### 2. Mostrar divergencia exponencial (Sensibilidad a C.I.)
initial_state_2 = [0.0, 1.0, 1.05 + 1e-8] # Ligerísima variación
trajectory_2 = odeint(lorenz_system, initial_state_2, t, args=(sigma, rho, beta))

plt.figure(figsize=(12, 5))
plt.plot(t, trajectory[:, 0], label='Trayectoria 1 (x0=1.05)', lw=1)
plt.plot(t, trajectory_2[:, 0], label='Trayectoria 2 (x0=1.05 + 1e-8)', linestyle='--', lw=1)
plt.title("Divergencia Exponencial de la Variable X (Efecto Mariposa)")
plt.xlabel("Tiempo")
plt.ylabel("X(t)")
plt.legend()
plt.xlim(0, 20)
plt.grid(True)
plt.show()

### 3. Proyección 2D y serie temporal de x(t)
plt.figure(figsize=(12, 5))
plt.plot(t, trajectory[:, 0], label='X(t)', lw=0.5)
plt.title("Serie Temporal de la Variable X (Parece Ruido)")
plt.xlabel("Tiempo")
plt.ylabel("X")
plt.grid(True)
plt.xlim(10, 30)
plt.show()

#### 3.2 Exponentes de Lyapunov: Midiendo el Caos

El exponente de Lyapunov ($\lambda$) mide la tasa de separación de trayectorias cercanas:

$$\delta(t) = \delta_0 e^{\lambda t}$$

**Interpretación:**

* $\lambda > 0$ $\rightarrow$ **Caos** (divergencia exponencial)
* $\lambda = 0$ $\rightarrow$ Periódico o cuasi-periódico
* $\lambda < 0$ $\rightarrow$ Convergente (punto fijo, atractor estable)

Para Lorenz: $\lambda \approx 0.9$ (**¡muy caótico!**)

---

In [ ]:
# [CÓDIGO 3: Cálculo de Exponentes de Lyapunov]

print("### Cálculo del Exponente de Lyapunov Mayor (MEL) ###")
print("Usaremos la librería 'nolds' para calcular el MEL de la serie de Lorenz. Se necesita: 'pip install nolds'\n")

# 1. Generar la serie de Lorenz (usando la función anterior)
sigma, rho, beta = 10, 28, 8/3
initial_state = [1.0, 1.0, 1.0]
t = np.arange(0, 100, 0.01) # Un poco más larga para la estimación
trajectory = odeint(lorenz_system, initial_state, t, args=(sigma, rho, beta))
lorenz_series = trajectory[5000:, 0] # Descartar transitorio inicial

# 2. Generar serie de Ruido Blanco (control)
white_noise = np.random.normal(size=len(lorenz_series))

# 3. Generar serie periódica (control)
sin_wave = np.sin(np.linspace(0, 100, len(lorenz_series)))

# 4. Calcular MEL (usando el algoritmo de Rosenstein)
try:
    lyap_lorenz = nolds.lyap_r(lorenz_series, emb_dim=3, lag=10)
    lyap_noise = nolds.lyap_r(white_noise, emb_dim=3, lag=1)
    lyap_periodic = nolds.lyap_r(sin_wave, emb_dim=3, lag=1)
    
    print(f"Exponente de Lyapunov (Lorenz): {lyap_lorenz:.3f} (Debe ser > 0 $\rightarrow$ CAOS)")
    print(f"Exponente de Lyapunov (Ruido Blanco): {lyap_noise:.3f} (Debe ser indeterminado o muy alto/sensible a parámetros)")
    print(f"Exponente de Lyapunov (Periódico): {lyap_periodic:.3f} (Debe ser $\approx 0$ $\rightarrow$ ORDEN)")

except ImportError:
    print("Error: La librería 'nolds' no está instalada. Ejecute: !pip install nolds")

### 4. Mackey–Glass: Una Ventana a los Fenómenos Biológicos

El sistema de Mackey–Glass (1977) se propuso para describir procesos fisiológicos con **retroalimentación retardada**, como la producción de células sanguíneas:

$$
\frac{dx}{dt} = \frac{\beta x(t-\tau)}{1 + x(t-\tau)^n} - \gamma x(t)
$$

**Parámetros típicos:**

* $\beta = 0.2$ (tasa de producción)
* $\gamma = 0.1$ (tasa de decaimiento)
* $n = 10$ (exponente no lineal)
* $\tau$ = retardo temporal (**parámetro crítico**)

**Régimen caótico:** Para $\tau > 16.8$, el sistema se vuelve caótico.

#### 4.1 ¿Por qué es Importante para ML?

* **Benchmark clásico** en predicción de series de tiempo no lineales.
* **Fenómeno común:** Retroalimentación retardada aparece en:
    * Regulación hormonal (insulina, cortisol)
    * Dinámica poblacional (depredador-presa)
    * Sistemas económicos (inflación, inversión)

* **Desafío técnico:** ARIMA, Exponential Smoothing y regresión lineal fallan completamente.

---

In [ ]:
# [CÓDIGO 4: Sistema Mackey-Glass]

print("### Implementación Numérica de la Ecuación de Retardo de Mackey-Glass (DDE) ###")

def mackey_glass(tau, n, beta, gamma, time_points):
    # Implementación simple con Euler (mejor usar 'pydelay' o 'ddesolvers' para mayor precisión)
    dt = 1.0
    history_len = tau * 10 # Asegurar suficiente historial
    x = deque([1.0] * history_len, maxlen=history_len) # Historial inicial
    result = []
    
    for i in range(time_points):
        # x_tau es el valor de x en t - tau
        x_tau = x[0]
        x_t = x[-1]
        
        # Ecuación de Mackey-Glass
        dxdt = (beta * x_tau) / (1 + x_tau**n) - gamma * x_t
        
        # Euler step
        new_x = x_t + dxdt * dt
        
        # Actualizar historial y resultado
        x.append(new_x)
        result.append(new_x)
        
    return np.array(result)

# Parámetros
beta = 0.2
gamma = 0.5
n = 10

time_points = 5000

# Generar series para diferentes tau
tau_orden = 12  # Orden (punto fijo o periódico)
tau_caos = 30  # Caos (tau > 16.8)

series_orden = mackey_glass(tau_orden, n, beta, gamma, time_points)
series_caos = mackey_glass(tau_caos, n, beta, gamma, time_points)

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(series_orden, lw=1)
plt.title(f"Mackey-Glass ($\tau={tau_orden}$) - Orden (Periódico)")

plt.subplot(1, 2, 2)
plt.plot(series_caos, lw=1)
plt.title(f"Mackey-Glass ($\tau={tau_caos}$) - Caos (Aperiódico)")

plt.tight_layout()
plt.show()

print("\nObservación: La serie caótica parece ruido, pero es 100% determinista. ¡Es un desafío para la predicción!")

#### 4.2 Aplicaciones Médicas Reales

Sistemas tipo Mackey-Glass describen:

* **Hematopoyesis** (producción de células sanguíneas) $\rightarrow$ Leucemia: desregulación del sistema.
* **Variabilidad del ritmo cardíaco (HRV)** $\rightarrow$ HRV normal: caótico (¡sano!). HRV regular: enfermedad (pérdida de complejidad).
* **Epilepsia** $\rightarrow$ Transición súbita a sincronización patológica.

**Contradicción médica:**

> En sistemas fisiológicos, **complejidad = salud**, **regularidad = enfermedad**.

(Lo opuesto a ingeniería, donde regularidad = funcionamiento correcto)

### 5. Mapas Caóticos: Complejidad desde la Simplicidad

A veces una sola ecuación discreta genera complejidad extrema.

#### 5.1 Mapa Logístico: El Caos en Una Dimensión

$$x_{t+1} = r x_t (1 - x_t)$$

**Diagrama de bifurcación:**

* $r < 3$: Convergencia a punto fijo
* $3 < r < 3.57$: Ciclos periódicos (periodo 2, 4, 8...)
* $r > 3.57$: Ruta al caos (cascada de bifurcaciones)
* $r = 4$: Caos completo

---

In [ ]:
# [CÓDIGO 5: Mapa Logístico - Diagrama de Bifurcación]

print("Diagrama de Bifurcación del Mapa Logístico")

N = 1000 # Iteraciones a descartar (transitorio)
M = 100  # Puntos a graficar por cada r
rs = np.linspace(2.5, 4.0, 5000) # Rango de r

x = 1e-5 * np.ones(len(rs)) # Condición inicial
points = []

for i in range(N + M):
    x = rs * x * (1 - x)
    if i >= N:
        points.append(x)

points = np.array(points)

plt.figure(figsize=(12, 7))
plt.plot(rs, points.T, ',k', alpha=0.25)
plt.title("Diagrama de Bifurcación del Mapa Logístico: Orden -> Caos")
plt.xlabel("Parámetro de crecimiento $r$")
plt.ylabel("Valor de $x_t$ después del transitorio")
plt.show()

print("Observación: Para r > 3.57 la ruta al caos es evidente (el eje y se 'llena').")

#### 5.2 Mapa de Hénon: Caos en 2D

$$
\begin{aligned}
x_{t+1} &= 1 - a x_t^2 + y_t \\
y_{t+1} &= b x_t
\end{aligned}
$$

Parámetros clásicos: $a = 1.4$, $b = 0.3$

**Propiedades:**

* Atractor con **estructura fractal** (autosimilar)
* Dimensión de correlación $\approx 1.26$ (**fractal!**)

#### 5.3 Mapa de Ikeda: Óptica Cuántica Caótica

$$
\begin{aligned}
x_{t+1} &= 1 + u(x_t \cos \theta_t - y_t \sin \theta_t) \\
y_{t+1} &= u(x_t \sin \theta_t + y_t \cos \theta_t) \\
\theta_t &= 0.4 - \frac{6}{1 + x_t^2 + y_t^2}
\end{aligned}
$$

Origen: Simula la luz dentro de una cavidad óptica con retroalimentación no lineal.

---

In [ ]:
# [CÓDIGO 6: Mapa de Hénon - Atractor 2D]

import numpy as np
import matplotlib.pyplot as plt

print("### Atractor de Hénon ###")

a = 1.4
b = 0.3
N = 20000
x = np.zeros(N)
y = np.zeros(N)

x[0], y[0] = 0.1, 0.1

for t in range(N - 1):
    x[t+1] = 1 - a * x[t]**2 + y[t]
    y[t+1] = b * x[t]

plt.figure(figsize=(8, 8))
plt.scatter(x[1000:], y[1000:], s=0.1, color='blue') # Descartar transitorio
plt.title("Atractor Extraño de Hénon")
plt.xlabel("$x_t$")
plt.ylabel("$y_t$")
plt.show()

In [ ]:
# [CÓDIGO 7: Mapa de Ikeda]

print("### Atractor del Mapa de Ikeda ###")

u = 0.9
N = 20000
x = np.zeros(N)
y = np.zeros(N)

x[0], y[0] = 0.1, 0.1

for t in range(N - 1):
    denom = 1 + x[t]**2 + y[t]**2
    theta_t = 0.4 - 6 / denom
    x[t+1] = 1 + u * (x[t] * np.cos(theta_t) - y[t] * np.sin(theta_t))
    y[t+1] = u * (x[t] * np.sin(theta_t) + y[t] * np.cos(theta_t))

plt.figure(figsize=(8, 8))
plt.scatter(x[1000:], y[1000:], s=0.1, color='red')
plt.title("Atractor del Mapa de Ikeda (Óptica No Lineal)")
plt.xlabel("$x_t$")
plt.ylabel("$y_t$")
plt.show()

#### 5.4 El Desafío para Machine Learning

Lo sorprendente de estos mapas:

A pesar de ser ecuaciones deterministas:

* ✅ Sus series parecen estocásticas
* ✅ Tienen autocorrelaciones complejas
* ✅ Presentan atractores fractales
* ✅ No se ajustan a ARIMA (no son lineales)
* ✅ Rompen supuesto gaussiano (colas pesadas, multimodalidad)
* ✅ Contienen ruido estructural (no es ruido de medición)

| Propiedad | Ruido Blanco | Sistema Caótico |
| :--- | :--- | :--- |
| Autocorrelación | Cero (lag > 0) | Decae rápido pero existe |
| Espectro de potencia | Plano | Estructura (no plano) |
| Dimensión de correlación | Infinita | Finita (fractal) |
| Predictibilidad | Cero | Corto plazo sí, largo no |
| Generación | Proceso aleatorio | Ecuación determinista |
| Sensibilidad inicial | No | Sí (exponencial) |

**Implicación:** Si un test dice "esto es ruido", no significa que lo sea. Puede ser caos determinista. Necesitas herramientas no lineales para distinguirlos.

---

In [ ]:
# [CÓDIGO 8: Ruido vs. Caos - ¿Cómo distinguirlos?]

print("### Dimensión de Correlación (D2) para distinguir Ruido de Caos Determinista ###")

# 1. Generar series (reutilizando las de Lorenz y Ruido)
N_points = 5000
lorenz_series = odeint(lorenz_system, [1.0, 1.0, 1.0], np.arange(0, N_points * 0.01, 0.01), args=(10, 28, 8/3))[1000:, 0]
white_noise = np.random.normal(size=len(lorenz_series))

print("Series Generadas (Lorenz: determinista; Ruido: estocástico)")

# 2. Reconstrucción de espacio de fase (Takens' embedding - visual)
def plot_takens(series, lag, dim, title):
    plt.figure(figsize=(6, 6))
    plt.scatter(series[:-lag], series[lag:], s=1, alpha=0.5)
    plt.title(title)
    plt.xlabel("$x_t$")
    plt.ylabel("$x_{t+\tau}$")
    plt.show()

plot_takens(lorenz_series, lag=10, dim=2, title="Embedding 2D - Serie de Lorenz (Estructura)")
plot_takens(white_noise, lag=1, dim=2, title="Embedding 2D - Ruido Blanco (Nube difusa)")

# 3. Cálculo de Dimensión de Correlación (D2)
try:
    D2_lorenz = nolds.corr_dim(lorenz_series, emb_dim=5, rvals=np.logspace(-2, 0, 15))
    D2_noise = nolds.corr_dim(white_noise, emb_dim=5, rvals=np.logspace(-2, 0, 15))
    
    print(f"\nDimensión de Correlación D2 (Lorenz): {D2_lorenz:.3f} (Debe ser finita, $\approx 2.05$ $\rightarrow$ CAOS)")
    print(f"Dimensión de Correlación D2 (Ruido): {D2_noise:.3f} (Debe ser cercana a la Dim. de Embedding $\rightarrow$ Ruido)")
    
    print("\nConclusión: Solo D2 (una herramienta no lineal) revela que Lorenz es determinista de baja dimensión.")

except ImportError:
    print("Error: La librería 'nolds' no está instalada. Ejecute: !pip install nolds")

## PARTE 2: SISTEMAS REALES COMPLEJOS

### 6. Por Qué Fallan los Métodos Tradicionales

Los métodos tradicionales no fallan por incompetencia, sino porque fueron diseñados para otro mundo.

#### 6.1 Los Supuestos Implícitos del ML Clásico

| Supuesto | Realidad en Sistemas Complejos |
| :--- | :--- |
| **Linealidad** | Interacciones no lineales dominantes |
| **Gaussianidad** | Colas pesadas, multimodalidad |
| **Estacionariedad** | Cambios de régimen, no estacionariedad estructural |
| **Baja sensibilidad** | Sensibilidad exponencial a inicialización (caos) |
| **Ruido aditivo** | Ruido multiplicativo, endógeno |

---

In [ ]:
# [CÓDIGO 9: Fracaso de Métodos Clásicos]

print("### Prueba de Fracaso: Predicción de Mackey-Glass con ARIMA y ETS ###")

# 1. Dataset: Serie de Mackey-Glass caótica (tau=30)
tau, n, beta, gamma = 30, 10, 0.2, 0.1
mg_series = mackey_glass(tau, n, beta, gamma, time_points=4000)[1000:3000] # 2000 puntos para estabilidad

train_size = 1800
test_size = 200
train, test = mg_series[:train_size], mg_series[train_size:]

# 2. Entrenar y predecir con ARIMA (ej. ARIMA(5,1,0))
try:
    model_arima = ARIMA(train, order=(5, 1, 0)).fit()
    pred_arima = model_arima.predict(start=train_size, end=train_size + test_size - 1)
    rmse_arima = np.sqrt(np.mean((pred_arima - test)**2))
except Exception as e:
    print(f"ARIMA falló: {e}")
    pred_arima = np.zeros(test_size)
    rmse_arima = np.nan

# 3. Entrenar y predecir con Exponential Smoothing (ETS)
try:
    model_ets = ExponentialSmoothing(train, seasonal='add', seasonal_periods=50, initialization_method='estimated').fit()
    pred_ets = model_ets.predict(start=train_size, end=train_size + test_size - 1)
    rmse_ets = np.sqrt(np.mean((pred_ets - test)**2))
except Exception as e:
    print(f"ETS falló: {e}")
    pred_ets = np.zeros(test_size)
    rmse_ets = np.nan

# 4. Visualización del Fracaso
plt.figure(figsize=(14, 6))
plt.plot(np.arange(train_size), train, label='Datos de Entrenamiento', color='blue')
plt.plot(np.arange(train_size, train_size + test_size), test, label='Realidad (Test)', color='red', linewidth=2)
plt.plot(np.arange(train_size, train_size + test_size), pred_arima, label=f'Predicción ARIMA (RMSE: {rmse_arima:.3f})', linestyle='--', color='orange')
plt.plot(np.arange(train_size, train_size + test_size), pred_ets, label=f'Predicción ETS (RMSE: {rmse_ets:.3f})', linestyle=':', color='green')
plt.title("Fracaso de Métodos Lineales/Estacionarios en Serie Caótica (Mackey-Glass)")
plt.xlabel("Paso de Tiempo")
plt.ylabel("X(t)")
plt.legend()
plt.show()

print("\nConclusión visual: Las predicciones colapsan rápidamente a un punto fijo o patrón simple, incapaces de seguir la dinámica caótica.")

### 7. Finanzas: Series Acopladas y Memoria Larga

Los mercados financieros no son ruido blanco (aunque académicos financieros durante décadas lo asumieron).

#### 7.1 Hechos Estilizados de Series Financieras

* **Heteroscedasticidad:** Varianza no constante (**Clusters de volatilidad**)
* **Colas pesadas:** Crashes más frecuentes que distribución normal predice.
* **Asimetría:** Caídas más abruptas que subidas (*leverage effect*)
* **No estacionariedad estructural:** Cambios de régimen (*bull/bear markets*)
* **Correlaciones de orden superior:** $E[r_t] \approx 0$, pero $E[|r_t|]$ tiene estructura (memoria larga en volatilidad).

#### 7.2 El Mito de la Eficiencia del Mercado

**Hipótesis de Mercado Eficiente (EMH):** "Los precios reflejan toda la información disponible, por lo que son impredecibles (*random walk*)."

**Realidad:** Los mercados son **sistemas adaptativos complejos**, no sistemas en equilibrio. Esto explica burbujas, crashes y la existencia de arbitraje estadístico exitoso (*quant funds*).

---

In [ ]:
# [CÓDIGO 10: Análisis de Serie Financiera Real]

print("Análisis de Retornos Diarios (S&P 500)")

# 1. Descargar datos (usando yfinance como ejemplo)
try:
    data = yf.download('^GSPC', start='2000-01-01', end='2024-01-01', auto_adjust=False)
    returns = data['Adj Close'].pct_change().dropna()

    # 2. Clusters de volatilidad (visualización)
    plt.figure(figsize=(14, 4))
    plt.plot(returns, lw=0.5)
    plt.title("Retornos Diarios S&P 500: Clusters de Volatilidad (Heteroscedasticidad)")
    plt.show()
    
    # 3. Test de Normalidad y Colas Pesadas (Q-Q Plot)
    plt.figure(figsize=(7, 7))
    stats.probplot(returns, dist="norm", plot=plt)
    plt.title("Q-Q Plot: Retornos vs. Distribución Normal (Colas Pesadas)")
    plt.show()
    
    # 4. Autocorrelación de |returns| (Memoria en Volatilidad)
    from statsmodels.graphics.tsaplots import plot_acf
    plot_acf(np.abs(returns), lags=50, title="ACF de |Retornos| (Evidencia de Memoria Larga)")
    plt.show()
    
    # 5. Test ADF para estacionariedad de los retornos (deben ser estacionarios)
    adf_result = adfuller(returns)
    print(f"\nTest ADF (Retornos): p-value = {adf_result[1]:.4f}")
    print("Si p-value < 0.05, los retornos son estacionarios. La complejidad reside en la varianza.")

except Exception as e:
    print(f"Error al descargar/procesar datos financieros (requiere 'pip install yfinance'): {e}")

### 8. Energía Eléctrica: Ciclos, Tendencias y Respuesta Colectiva

Los sistemas eléctricos muestran complejidad organizada:

#### 8.1 Características de Series de Demanda Eléctrica

* **Multiscale:** Ciclo diario, semanal, anual.
* **No linealidad:** Efectos de saturación, respuesta no lineal a la temperatura.
* **Acoplamiento social:** Consumo humano altamente sincronizado (todos cocinan a las 7 PM).
* **Eventos extremos:** Olas de calor $\rightarrow$ picos de demanda; fallas en cascada (*blackouts*).

#### 8.2 Desafío de Predicción

Se necesita un modelo robusto que maneje múltiples estacionalidades superpuestas, no linealidad (temperatura), y la capacidad de predecir a múltiples horizontes.

---

In [ ]:
# [CÓDIGO 11: Serie de Demanda Eléctrica - Descomposición STL]

print("### Descomposición Multi-Escala de una Serie de Demanda Eléctrica (Simulada) ###")

# 1. Generar datos simulados con múltiples estacionalidades y no linealidad
time = np.arange(0, 365 * 24) / 24.0 # 2 años de datos horarios
trend = 0.5 * time + 50
daily_cycle = 20 * np.sin(2 * np.pi * time) + 10 * np.sin(4 * np.pi * time) # Dos picos diarios
annual_cycle = 40 * np.sin(2 * np.pi * time / 365.0) # Estacionalidad anual (verano/invierno)
noise = np.random.normal(0, 5, len(time))
demand = trend + daily_cycle + annual_cycle + noise
demand_series = pd.Series(demand, index=pd.to_datetime('2023-01-01') + pd.to_timedelta(np.arange(len(time)), unit='h'))

# 2. Descomposición STL
print("Realizando Descomposición STL...")
# La demanda eléctrica tiene estacionalidad diaria (24) y semanal (24*7=168)
stl_result = STL(demand_series, period=168, robust=True).fit()

# 3. Visualización
plt.figure(figsize=(10, 8))
stl_result.plot()
plt.suptitle("Descomposición STL de Demanda Eléctrica (Tendencia, Estacionalidad, Residuo)", y=1.02)
plt.show()

print("\nObservación: El residuo (remnant) aún contiene estructura no lineal no capturada por el modelo lineal de tendencia y estacionalidad.")

### 9. Neurología: EEG y Epilepsia como Sistemas Dinámicos

Las señales electroencefalográficas (EEG) son el ejemplo paradigmático de complejidad biológica.

#### 9.1 Propiedades del EEG

* **No estacionario extremo:** Transiciones abruptas (microsueños, estados mentales).
* **Sincronización patológica:** Actividad normal es **caótica** (saludable). Epilepsia es **hipersincronización súbita** (pérdida de complejidad).
* **Multi-canal:** Acoplamiento espacial (redes funcionales).

#### 9.2 Detección de Epilepsia como Problema de Clasificación

El desafío es predecir una convulsión (*pre-ictal*) antes de que ocurra, utilizando *features no lineales* críticos (entropía, dimensión de correlación) que distinguen el caos saludable del orden patológico.

### Detección de Transición (Pre-Ictal $\rightarrow$ Ictal) mediante Entropía de Muestra


In [ ]:
# [CÓDIGO 12: Análisis de EEG Epiléptico (Simulado) - Entropía]


# 1. Simulación de señal EEG (Transición: Caos $\rightarrow$ Orden)
N = 3000
noise_normal = np.random.randn(N) * 0.5 + np.sin(np.arange(N) * 0.1) # Actividad 'caótica' normal
spike_onset = 2000
ictal_burst = np.sin(np.arange(N - spike_onset) * 0.5) * 5 # Sincronización patológica (orden)
eeg_signal = np.concatenate([noise_normal[:spike_onset], ictal_burst + noise_normal[spike_onset:] * 0.1])

# 2. Visualización de la señal cruda
plt.figure(figsize=(14, 4))
plt.plot(eeg_signal, lw=0.5)
plt.axvline(x=spike_onset, color='red', linestyle='--', label='Inicio Ictal')
plt.title("Señal EEG Simulada: Transición Caótica -> Ordenada (Epilepsia)")
plt.show()

# 3. Entropía de Muestra (Sample Entropy - SampEn) por ventana
window_size = 250
step = 50
sampen_values = []
for i in range(0, len(eeg_signal) - window_size, step):
    window = eeg_signal[i:i + window_size]
    try:
        # El valor de SampEn BAJA cuando la complejidad (caos) se pierde
        sampen = nolds.sampen(window, emb_dim=2, tolerance=0.2 * np.std(window))
        sampen_values.append(sampen)
    except:
        sampen_values.append(np.nan)

plt.figure(figsize=(14, 4))
plt.plot(np.arange(len(sampen_values)) * step + window_size / 2, sampen_values, 'o-', color='blue')
plt.axvline(x=spike_onset, color='red', linestyle='--', label='Inicio Ictal')
plt.title("Entropía de Muestra (SampEn): Pérdida de Complejidad en la Transición")
plt.xlabel("Tiempo")
plt.ylabel("SampEn (Complejidad)")
plt.show()

print("\nObservación: La caída en SampEn indica que la señal se ha vuelto más regular/sincronizada, lo que es un indicador de patología en este contexto.")

### 10. Señales Fisiológicas: Corazón, Respiración, Movimiento

El corazón humano es un oscilador caótico (sí, ¡el caos es saludable!).

#### 10.1 Variabilidad del Ritmo Cardíaco (HRV)

| Condición | HRV | Interpretación |
| :--- | :--- | :--- |
| Persona sana | Alta variabilidad | Sistema adaptativo, **caótico** |
| Enfermedad cardíaca | Baja variabilidad | **Pérdida de complejidad** |
| Antes de infarto | HRV disminuye | Señal de alarma |

**Análisis no lineal de HRV:** Gráfico de Poincaré, Entropía aproximada (ApEn), Entropía de muestra (SampEn), Dimensión de correlación.

---

In [ ]:
# [CÓDIGO 13: Análisis de HRV - Gráfico de Poincaré]

print("### Gráfico de Poincaré para Variabilidad del Ritmo Cardíaco (HRV) ###")

# 1. Simulación de RR intervals (tiempo entre latidos)
# HRV Sano (Caótico): Media estable con alta varianza (elipse difusa)
rr_sano = np.random.normal(loc=1000, scale=10, size=500) + 10 * np.sin(np.arange(500) * 0.1)
# HRV Enfermo (Regular): Media estable con baja varianza (punto/línea compacta)
rr_enfermo = np.random.normal(loc=1000, scale=2, size=500)

# 2. Gráfico de Poincaré (RR_n vs. RR_{n+1})
def plot_poincare(rr_intervals, title):
    plt.figure(figsize=(6, 6))
    plt.scatter(rr_intervals[:-1], rr_intervals[1:], s=5, alpha=0.6)
    plt.axline((1000, 1000), slope=1, color='red', linestyle='--')
    plt.title(title)
    plt.xlabel("$RR_n$ (ms)")
    plt.ylabel("$RR_{n+1}$ (ms)")
    plt.axis('equal')
    plt.grid(True)
    plt.show()

plot_poincare(rr_sano, "HRV Sano (Alta Complejidad - Elipse Ancha)")
plot_poincare(rr_enfermo, "HRV Enfermo (Baja Complejidad - Punto Compacto)")

print("\nConclusión: Una elipse ancha (mayor área) en el Gráfico de Poincaré indica un sistema más complejo/caótico, lo cual es un signo de salud en el corazón.")

## PARTE 3: SÍNTESIS Y CONCLUSIONES

### 11. Anatomía de un Fracaso: Por Qué Fallan los Modelos Tradicionales

Los métodos tradicionales asumen un mundo dócil:

| Supuesto | Mundo Dócil (Asumido) | Mundo Real (Observado) |
| :--- | :--- | :--- |
| **Linealidad** | $y = mx + b$ | $y = f(x_1, \dots, x_n)$ con $f$ no lineal |
| **Ruido** | Aditivo gaussiano | Multiplicativo, endógeno, estructural |
| **Estacionariedad** | Media y varianza constantes | Cambios de régimen, tendencias |
| **Sensibilidad** | Pequeños errores $\rightarrow$ pequeños efectos | Sensibilidad exponencial (caos) |
| **Mínimos** | Un solo óptimo global | Múltiples mínimos locales |

---

In [ ]:
# [CÓDIGO 14: Mundo Dócil vs. Mundo Real - El Contraste Visual]

print("### El Contraste Visual: Modelado Lineal vs. Realidad No Lineal ###")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Mundo Dócil vs. Mundo Real', fontsize=16)

# Panel 1: Función lineal con ruido gaussiano
X1 = np.linspace(0, 10, 100).reshape(-1, 1)
y1 = 2 * X1 + 5 + np.random.normal(0, 1, 100).reshape(-1, 1)
lr1 = LinearRegression().fit(X1, y1)
axes[0, 0].scatter(X1, y1, s=10)
axes[0, 0].plot(X1, lr1.predict(X1), color='red')
axes[0, 0].set_title("1. Mundo Dócil: Linealidad Funciona")

# Panel 2: Función no lineal (sin²(x)) con ruido
X2 = np.linspace(0, 10, 100).reshape(-1, 1)
y2 = 5 * np.sin(X2)**2 + np.random.normal(0, 0.5, 100).reshape(-1, 1)
lr2 = LinearRegression().fit(X2, y2)
axes[0, 1].scatter(X2, y2, s=10)
axes[0, 1].plot(X2, lr2.predict(X2), color='red')
axes[0, 1].set_title("2. Mundo Real: No Linealidad (Regresión Lineal Falla)")

# Panel 3: Serie de Lorenz (caótica) y su predicción 'lineal' (ARIMA)
lorenz_series = odeint(lorenz_system, [1.0, 1.0, 1.0], np.arange(0, 30, 0.01), args=(10, 28, 8/3))[1000:, 0]
train_l = lorenz_series[:1500]
test_l = lorenz_series[1500:2000]
try:
    model_arima_l = ARIMA(train_l, order=(2, 0, 0)).fit()
    pred_arima_l = model_arima_l.predict(start=1500, end=2000-1)
except Exception:
    pred_arima_l = np.ones(len(test_l)) * np.mean(train_l)

axes[1, 0].plot(np.arange(len(lorenz_series[:2000])), lorenz_series[:2000], lw=0.5, color='blue', label='Real')
axes[1, 0].plot(np.arange(1500, 2000), pred_arima_l, color='red', linestyle='--', label='Pred. ARIMA')
axes[1, 0].axvline(x=1500, color='grey', linestyle=':')
axes[1, 0].set_title("3. Mundo Real: Caos Determinista (ARIMA Colapsa)")

# Panel 4: Serie Financiera Real (S&P 500) - Colas pesadas
returns_sample = np.concatenate([np.random.normal(0, 1, 100), np.random.normal(0, 5, 20)]) # Normal + 20 outliers
axes[1, 1].hist(returns_sample, bins=30, density=True, alpha=0.7, label='Empírica')
xmin, xmax = axes[1, 1].get_xlim()
x_norm = np.linspace(xmin, xmax, 100)
p_norm = stats.norm.pdf(x_norm, np.mean(returns_sample), np.std(returns_sample))
axes[1, 1].plot(x_norm, p_norm, 'r--', label='Gaussiana (Asumida)')
axes[1, 1].set_title("4. Mundo Real: Colas Pesadas (No Gaussianidad)")
axes[1, 1].legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### 12. La Necesidad de Metaheurísticas: Un Argumento Estructural

Esta libreta es un argumento estructural para justificar el curso completo:

#### 12.1 Si el Mundo es Complejo, Necesitamos Herramientas Complejas

**Silogismo:**

1.  **Premisa 1:** Los datos reales provienen de sistemas complejos.
2.  **Premisa 2:** Los sistemas complejos violan supuestos de métodos clásicos.
3.  **Premisa 3:** Los métodos clásicos fallan cuando sus supuestos se violan.
4.  **Conclusión:** Necesitamos métodos que no asuman linealidad, convexidad ni estacionariedad.

**Metaheurísticas cumplen esto porque:**

* ✅ No requieren gradientes (funcionan en espacios discretos y rugosos)
* ✅ Exploran globalmente (resisten mínimos locales)
* ✅ Son flexibles (se adaptan a cualquier función objetivo no lineal)
* ✅ Manejan multimodalidad (poblaciones diversas)

#### 12.2 Tabla Comparativa Final

| Método | Supuestos Clave | Funciona en Sistemas Complejos |
| :--- | :--- | :--- |
| Regresión Lineal | Linealidad, Gaussianidad | ❌ No |
| ARIMA | Estacionariedad, Linealidad | ❌ No |
| Gradient Descent | Diferenciabilidad, Convexidad | ❌ No |
| Redes Neuronales | Datos abundantes, suavidad | ⚠️ Parcial (sobreajuste, caja negra) |
| **Metaheurísticas** | **Ninguno (evaluación de función)** | **✅ Sí** |

### 13. Roadmap del Curso: De la Complejidad a las Soluciones

Esta libreta establece el **por qué**. El resto del curso muestra el **cómo**.

| Sistema Complejo | Desafío | Metaheurística Apropiada | Módulo |
| :--- | :--- | :--- | :--- |
| Lorenz, Mackey-Glass | Predicción no lineal | Programación Genética para regresión simbólica | Módulo 4 |
| Series financieras | Múltiples óptimos | PSO, DE para hiperparámetros | Módulos 5-6 |
| EEG, señales | Feature selection | Algoritmos Genéticos (GA) con *wrapper* | Módulo 3 |

---

## ACTIVIDADES PRÁCTICAS

### Actividad 1: Experimenta con el Caos

**Desafío:**

Genera series del mapa logístico con $r = 2.5, 3.3, 3.6, 4.0$. Para cada serie, calcula: Autocorrelación, Histograma, Exponente de Lyapunov (aproximado).

**Pregunta:** ¿En cuál valor de $r$ comienza el caos?

### Actividad 2: ¿Ruido o Caos?

**Desafío:**

Te doy dos series de 1000 puntos. Una es ruido blanco, la otra es el sistema de Lorenz (proyección de x(t)).

1.  Usando solo visualización, ¿puedes distinguirlas?
2.  Calcula dimensión de correlación con Takens' embedding.
3.  La que tenga dimensión finita es el sistema caótico.

### Actividad 3: Predecir lo Impredecible

**Desafío:**

Genera 500 puntos de Mackey-Glass ($\tau=17$). Entrena: ARIMA (usa `auto_arima` si está disponible o un orden fijo), Random Forest con lags manuales, y un LSTM simple.

Compara RMSE en test set (últimos 100 puntos). ¿Cuál método falla menos?

---

In [ ]:
# [ACTIVIDAD 1 - Tu código aquí: Mapa Logístico]

# Se requieren las implementaciones de: Generación de mapa, ACF, Histograma y nolds.lyap_r

In [ ]:
# [ACTIVIDAD 2 - Tu código aquí: Ruido vs. Caos]

print("Serie A: Generar ruido blanco")
print("Serie B: Generar serie de Lorenz")
print("El estudiante debe aplicar nolds.corr_dim() a ambas.")

In [ ]:
# [ACTIVIDAD 3 - Tu código aquí: Predicción de Mackey-Glass]

# Se requiere scikit-learn (Random Forest) y TensorFlow/Keras (LSTM)

<div style="border: 2px solid #24398A; border-radius: 15px; padding: 20px; background-color: #fcfcfc;">
    <h3 style="color: #24398A; margin-top: 0;">Conclusiones: Dinámicas No Lineales y Emergencia</h3>
    <p style="font-size: 15px; line-height: 1.6;">
        Hemos abordado la complejidad no como una medida de dificultad, sino como una propiedad intrínseca de los sistemas donde el todo es mayor que la suma de sus partes. Este enfoque es crucial para modelar fenómenos económicos, sociales y biológicos con fidelidad.
    </p>
    <ul style="font-size: 14px; color: #333;">
        <li><b>Criterio de Modelado:</b> El reconocimiento de la auto-organización y la sensibilidad a condiciones iniciales permite anticipar comportamientos que los modelos lineales ignoran.</li>
        <li><b>Fundamentación Técnica:</b> Las metaheurísticas se justifican formalmente como las herramientas idóneas para navegar la rugosidad de estos sistemas.</li>
    </ul>

</div>